In [1]:
# tf ->2.19.0 | 2.18.0
!pip install -U -q "tf-models-official"
# !pip show tf-models-official

In [2]:
# !pip install keras==3.5.0
!pip uninstall keras -y

# !pip uninstall tensorflow -y
# !pip install tensorflow==2.17.1

!pip install -U tf_keras # Keras 2
import os
os.environ["TF_USE_LEGACY_KERAS"] = "1"

Found existing installation: keras 3.9.2
Uninstalling keras-3.9.2:
  Successfully uninstalled keras-3.9.2
  Using cached keras-3.9.2-py3-none-any.whl.metadata (6.1 kB)
Using cached keras-3.9.2-py3-none-any.whl (1.3 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-decision-forests 1.11.0 requires tensorflow==2.18.0, but you have tensorflow 2.19.0 which is incompatible.
dopamine-rl 4.1.2 requires gymnasium>=1.0.0, but you have gymnasium 0.29.0 which is incompatible.


# START OF LATEST CODE

In [3]:
import os
import gc
import cv2
import pickle
import numpy as np
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
from sklearn.utils import class_weight
from tensorflow.keras import mixed_precision
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from official.projects.movinet.modeling import movinet, movinet_model
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.metrics import confusion_matrix, classification_report, ConfusionMatrixDisplay

2025-04-18 21:32:20.982792: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1745011941.006644   68357 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1745011941.013873   68357 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1745011941.034936   68357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745011941.034954   68357 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1745011941.034957   68357 computation_placer.cc:177] computation placer alr

In [4]:
# Disable mixed precision temporarily
mixed_precision.set_global_policy('float32')

In [5]:
def get_class_names(data_dir):
    class_names = sorted([folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))])
    print(f"Class names in {data_dir}:", class_names)
    return class_names

# Check all datasets
train_class_names = get_class_names("/kaggle/input/capstone-dataset-updated/split_data_2/train")
val_class_names = get_class_names("/kaggle/input/capstone-dataset-updated/split_data_2/val")
test_class_names = get_class_names("/kaggle/input/capstone-dataset-updated/split_data_2/test")

# Ensure consistency
assert train_class_names == val_class_names == test_class_names, "Class names mismatch across datasets!"
class_names = train_class_names
num_classes = len(class_names)
print("Number of classes:", num_classes)

Class names in /kaggle/input/capstone-dataset-updated/split_data_2/train: ['Corner', 'Direct free-kick', 'Foul', 'Goal', 'Indirect free-kick', 'Kick-off', 'Offside', 'Shots off target', 'Shots on target', 'Substitution', 'Throw-in', 'Yellow card']
Class names in /kaggle/input/capstone-dataset-updated/split_data_2/val: ['Corner', 'Direct free-kick', 'Foul', 'Goal', 'Indirect free-kick', 'Kick-off', 'Offside', 'Shots off target', 'Shots on target', 'Substitution', 'Throw-in', 'Yellow card']
Class names in /kaggle/input/capstone-dataset-updated/split_data_2/test: ['Corner', 'Direct free-kick', 'Foul', 'Goal', 'Indirect free-kick', 'Kick-off', 'Offside', 'Shots off target', 'Shots on target', 'Substitution', 'Throw-in', 'Yellow card']
Number of classes: 12


In [6]:
# # Video loading function with RGB conversion and additional augmentations
# def load_video(video_path, max_frames=16, target_size=(224, 224), augment=False, frame_stride=1):
#     video_path = video_path.numpy().decode("utf-8")
#     cap = cv2.VideoCapture(video_path)
#     frames = []
#     frame_count = 0
#     while len(frames) < max_frames:
#         ret, frame = cap.read()
#         if not ret or frame is None:
#             break
#         if frame_count % frame_stride == 0:
#             frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#             if augment:
#                 if np.random.rand() < 0.5:
#                     h, w = frame.shape[:2]
#                     crop_h, crop_w = int(h * 0.9), int(w * 0.9)
#                     y = np.random.randint(0, h - crop_h)
#                     x = np.random.randint(0, w - crop_w)
#                     frame = frame[y:y+crop_h, x:x+crop_w]
#                 if np.random.rand() < 0.3:
#                     angle = np.random.uniform(-10, 10)
#                     M = cv2.getRotationMatrix2D((frame.shape[1]/2, frame.shape[0]/2), angle, 1)
#                     frame = cv2.warpAffine(frame, M, (frame.shape[1], frame.shape[0]))
#             frame = cv2.resize(frame, target_size)
#             frame = frame / 255.0
#             if augment:
#                 if np.random.rand() < 0.5:
#                     frame = cv2.flip(frame, 1)
#                 if np.random.rand() < 0.5:
#                     factor = 0.8 + 0.4 * np.random.rand()
#                     frame = np.clip(frame * factor, 0, 1)
#             frames.append(frame)
#         frame_count += 1
#     cap.release()
#     while len(frames) < max_frames:
#         frames.append(np.zeros(target_size + (3,), dtype=np.float32))
#     video = np.array(frames, dtype=np.float32)
#     if np.any(np.isnan(video)):
#         raise ValueError(f"NaN values found in video: {video_path}")
#     return tf.convert_to_tensor(video)

def load_video(video_path, max_frames=16, target_size=(224, 224), augment=False, frame_stride=2):
    try:
        video_path = video_path.numpy().decode("utf-8")
        cap = cv2.VideoCapture(video_path)
        if not cap.isOpened():
            print(f"Error: Cannot open video: {video_path}")
            return tf.zeros([max_frames, target_size[0], target_size[1], 3], dtype=tf.float32)
        frames = []
        frame_count = 0
        valid_frames = 0
        while len(frames) < max_frames:
            ret, frame = cap.read()
            if not ret or frame is None:
                break
            if frame_count % frame_stride == 0:
                frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                if augment and np.random.rand() < 0.5:  # Reduced augmentation frequency
                    if np.random.rand() < 0.6:
                        h, w = frame.shape[:2]
                        crop_h, crop_w = int(h * 0.9), int(w * 0.9)
                        y = np.random.randint(0, h - crop_h)
                        x = np.random.randint(0, w - crop_w)
                        frame = frame[y:y+crop_h, x:x+crop_w]
                frame = cv2.resize(frame, target_size)
                frame = frame / 255.0
                frames.append(frame)
                valid_frames += 1
            frame_count += 1
        cap.release()
        while len(frames) < max_frames:
            frames.append(np.zeros(target_size + (3,), dtype=np.float32))
        video = np.array(frames, dtype=np.float32)
        if valid_frames == 0:
            print(f"Warning: No valid frames in video: {video_path}")
            return tf.zeros([max_frames, target_size[0], target_size[1], 3], dtype=tf.float32)
        if np.any(np.isnan(video)) or np.all(video == 0):
            print(f"Error: Invalid video (NaN or all-zero): {video_path}")
            return tf.zeros([max_frames, target_size[0], target_size[1], 3], dtype=tf.float32)
        return tf.convert_to_tensor(video)
    except Exception as e:
        print(f"Error processing video {video_path}: {str(e)}")
        return tf.zeros([max_frames, target_size[0], target_size[1], 3], dtype=tf.float32)


def get_class_names(data_dir):
    class_names = sorted([folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))])
    print(class_names)
    return class_names

# def create_dataset(directory, batch_size, class_names, augment=False):
#     video_paths, labels = [], []
#     for idx, class_name in enumerate(class_names):
#         class_dir = os.path.join(directory, class_name)
#         for f in os.listdir(class_dir):
#             if f.endswith(".mp4"):
#                 video_paths.append(os.path.join(class_dir, f))
#                 labels.append(idx)
#     dataset = tf.data.Dataset.from_tensor_slices((video_paths, labels))
#     def _parse_function(filename, label):
#         video = tf.py_function(lambda f: load_video(f, max_frames=16, target_size=(224, 224), augment=augment, frame_stride=2), [filename], tf.float32)
#         video.set_shape([16, 224, 224, 3])
#         return video, label
#     dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
#     if augment:
#         dataset = dataset.shuffle(100)
#     dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
#     return dataset

def create_dataset(directory, batch_size, class_names, augment=False, frame_stride=2):
    video_paths, labels = [], []
    for idx, class_name in enumerate(class_names):
        class_dir = os.path.join(directory, class_name)
        if os.path.exists(class_dir):
            for f in os.listdir(class_dir):
                if f.endswith(".mp4"):
                    video_paths.append(os.path.join(class_dir, f))
                    labels.append(idx)
    dataset = tf.data.Dataset.from_tensor_slices((video_paths, labels))
    def _parse_function(filename, label):
        video = tf.py_function(
            lambda f: load_video(f, max_frames=16, target_size=(224, 224), augment=augment, frame_stride=frame_stride),
            [filename],
            tf.float32
        )
        video.set_shape([16, 224, 224, 3])
        return video, label
    dataset = dataset.shuffle(2000 if augment else 1000).cache().prefetch(tf.data.AUTOTUNE)  # Added cache and prefetch
    dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset, video_paths, labels

# def create_dataset_from_paths(video_paths, labels, batch_size, class_names, augment=False, frame_stride=2):
#     dataset = tf.data.Dataset.from_tensor_slices((video_paths, labels))
#     def _parse_function(filename, label):
#         video = tf.py_function(
#             lambda f: load_video(f, max_frames=16, target_size=(224, 224), augment=augment, frame_stride=frame_stride),
#             [filename],
#             tf.float32
#         )
#         video.set_shape([16, 224, 224, 3])
#         return video, label
#     dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
#     if augment:
#         dataset = dataset.shuffle(100)
#     dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
#     return dataset


# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$


# def create_dataset_from_paths(video_paths, labels, batch_size, class_names, augment=False, frame_stride=2):
#     dataset = tf.data.Dataset.from_tensor_slices((video_paths, labels))
#     def _parse_function(filename, label):
#         video = tf.py_function(
#             lambda f: load_video(f, max_frames=16, target_size=(224, 224), augment=augment, frame_stride=frame_stride),
#             [filename],
#             tf.float32
#         )
#         video.set_shape([16, 224, 224, 3])
#         return video, label
#     dataset = dataset.shuffle(buffer_size=len(video_paths), reshuffle_each_iteration=True)  # Fixed shuffling
#     dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)
#     dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
#     dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
#     return dataset

# $$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$$

def create_dataset_from_paths(video_paths, labels, batch_size, class_names, augment=False, frame_stride=2):
    dataset = tf.data.Dataset.from_tensor_slices((video_paths, labels))
    def _parse_function(filename, label):
        video = tf.py_function(
            lambda f: load_video(f, max_frames=16, target_size=(224, 224), augment=augment, frame_stride=frame_stride),
            [filename],
            tf.float32
        )
        video.set_shape([16, 224, 224, 3])
        return video, label
    dataset = dataset.shuffle(buffer_size=len(video_paths), reshuffle_each_iteration=True)
    dataset = dataset.cache().prefetch(tf.data.AUTOTUNE)
    dataset = dataset.map(_parse_function, num_parallel_calls=tf.data.AUTOTUNE)
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

def get_merged_dataset(train_dir, val_dir, test_dir, class_names):
    video_paths, labels = [], []
    for idx, class_name in enumerate(class_names):
        # Training directory
        train_class_dir = os.path.join(train_dir, class_name)
        if os.path.exists(train_class_dir):
            for f in os.listdir(train_class_dir):
                if f.endswith(".mp4"):
                    video_paths.append(os.path.join(train_class_dir, f))
                    labels.append(idx)
        # Validation directory
        val_class_dir = os.path.join(val_dir, class_name)
        if os.path.exists(val_class_dir):
            for f in os.listdir(val_class_dir):
                if f.endswith(".mp4"):
                    video_paths.append(os.path.join(val_class_dir, f))
                    labels.append(idx)
        # Test directory
        test_class_dir = os.path.join(test_dir, class_name)
        if os.path.exists(test_class_dir):
            for f in os.listdir(test_class_dir):
                if f.endswith(".mp4"):
                    video_paths.append(os.path.join(test_class_dir, f))
                    labels.append(idx)
    print(f"Total merged dataset: {len(video_paths)} clips")
    print("Class distribution:", {class_names[i]: sum(np.array(labels) == i) for i in range(len(class_names))})
    return video_paths, labels


# def visualize_clips(dataset, class_names, paths, num_clips=4):
#     for videos, labels in dataset.take(1):
#         indices = tf.range(len(labels)).numpy()[:num_clips]
#         print("Sample validation labels:", labels.numpy()[:num_clips])
#         print("Sample validation paths:", [paths[i] for i in indices])
#         plt.figure(figsize=(12, 3))
#         for i in range(min(num_clips, videos.shape[0])):
#             plt.subplot(1, num_clips, i+1)
#             plt.imshow(videos[i][0])
#             plt.title(f"Label: {class_names[labels[i]]}")
#             plt.axis('off')
#         plt.show()


def visualize_clips(dataset, class_names, paths, num_clips=4):
    for videos, labels in dataset.take(1):
        indices = tf.range(len(labels)).numpy()[:num_clips]
        print("Sample validation labels:", labels.numpy()[:num_clips])
        print("Sample validation paths:", [paths[i] for i in indices])
        plt.figure(figsize=(12, 3))
        for i in range(min(num_clips, videos.shape[0])):
            plt.subplot(1, num_clips, i+1)
            plt.imshow(videos[i][0])
            plt.title(f"Label: {class_names[labels[i].numpy()]}")
            plt.axis('off')
        plt.show()

def save_metrics(fold_metrics, filename='fold_metrics.pkl'):
    with open(filename, 'wb') as f:
        pickle.dump(fold_metrics, f)

def load_metrics(filename='fold_metrics.pkl'):
    if os.path.exists(filename):
        with open(filename, 'rb') as f:
            return pickle.load(f)
    return []

In [7]:
train_dir = "/kaggle/input/capstone-dataset-updated/split_data_2/train"
val_dir = "/kaggle/input/capstone-dataset-updated/split_data_2/val"
test_dir = "/kaggle/input/capstone-dataset-updated/split_data_2/test"

class_names = get_class_names(train_dir)
num_classes = len(class_names)
print("Number of classes:", num_classes)

all_paths, all_labels = get_merged_dataset(train_dir, val_dir, test_dir, class_names)
all_paths = np.array(all_paths)
all_labels = np.array(all_labels)

['Corner', 'Direct free-kick', 'Foul', 'Goal', 'Indirect free-kick', 'Kick-off', 'Offside', 'Shots off target', 'Shots on target', 'Substitution', 'Throw-in', 'Yellow card']
Number of classes: 12
Total merged dataset: 3694 clips
Class distribution: {'Corner': 231, 'Direct free-kick': 315, 'Foul': 264, 'Goal': 333, 'Indirect free-kick': 259, 'Kick-off': 352, 'Offside': 267, 'Shots off target': 348, 'Shots on target': 357, 'Substitution': 337, 'Throw-in': 267, 'Yellow card': 364}


In [8]:
# k = 1
# skf = StratifiedKFold(n_splits=k, shuffle=True, random_state=42)
fold_metrics = load_metrics()
batch_size = 5
num_frames = 16
resolution = 224
num_epochs = 6
num_train = len(all_paths)  # ~3148
train_steps = num_train // batch_size
total_train_steps = train_steps * num_epochs

# for fold, (train_idx, val_idx) in enumerate(skf.split(all_paths, all_labels)):
#     if any(m['fold'] == fold + 1 for m in fold_metrics):
#         print(f"Skipping Fold {fold + 1}: Already completed")
#         continue
        
#     print(f"\nFold {fold + 1}/{k}")
    
#     # Split data
#     train_paths_fold = all_paths[train_idx]
#     train_labels_fold = all_labels[train_idx]
#     val_paths_fold = all_paths[val_idx]
#     val_labels_fold = all_labels[val_idx]
    
#     print(f"Training clips: {len(train_paths_fold)}")
#     print(f"Validation clips: {len(val_paths_fold)}")
#     print("Validation class distribution:", {class_names[i]: sum(val_labels_fold == i) for i in range(num_classes)})
#     print("Validation labels (all):", val_labels_fold.tolist())
#     print("Unique validation labels:", np.unique(val_labels_fold))
    
#     # Create datasets
#     train_data = create_dataset_from_paths(
#         train_paths_fold, train_labels_fold, batch_size, class_names, augment=True, frame_stride=2
#     )
#     val_data = create_dataset_from_paths(
#         val_paths_fold, val_labels_fold, batch_size, class_names, augment=False, frame_stride=2
#     )

#     print(f"Visualizing validation clips for fold {fold + 1}")
#     visualize_clips(val_data, class_names, val_paths_fold)
    
#     # Compute class weights
#     class_weights = class_weight.compute_class_weight(
#         'balanced', classes=np.unique(train_labels_fold), y=train_labels_fold
#     )
#     class_weight_dict = dict(enumerate(class_weights))
#     if 11 in class_weight_dict:  # Reduce Yellow card weight
#         class_weight_dict[11] *= 0.5
#     print(f"Class weights for fold {fold + 1}:", class_weight_dict)
    
#     # Model setup
#     tf.keras.backend.clear_session()
#     gc.collect()
#     # backbone = movinet.Movinet(model_id='a3')
#     # backbone.build([1, num_frames, resolution, resolution, 3])
    
#     # # # Load pretrained weights for backbone
#     # # !wget https://storage.googleapis.com/tf_model_garden/vision/movinet/movinet_a3_base.tar.gz -O movinet_a3_base.tar.gz -q
#     # # !tar -xvf movinet_a3_base.tar.gz
#     # # checkpoint_dir = 'movinet_a3_base'
#     # # checkpoint_path = tf.train.latest_checkpoint(checkpoint_dir)
#     # # checkpoint = tf.train.Checkpoint(backbone=backbone)
#     # # status = checkpoint.restore(checkpoint_path).expect_partial()
#     # # status.assert_nontrivial_match()  # Verify some weights were loaded
    
#     # # Build full model
#     # model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=num_classes, dropout_rate=0.5)
#     # model.build([batch_size, num_frames, resolution, resolution, 3])
    
#     # # Build classifier with partial backbone unfreezing
#     # def build_classifier(backbone, num_classes, freeze_backbone=False):
#     #     model = movinet_model.MovinetClassifier(
#     #         backbone=backbone,
#     #         num_classes=num_classes,
#     #         dropout_rate=0.5
#     #     )
#     #     model.build([batch_size, num_frames, resolution, resolution, 3])
#     #     if freeze_backbone:
#     #         for layer in model.layers[:-1]:
#     #             layer.trainable = False
#     #     else:
#     #         for layer in model.layers:
#     #             layer.trainable = True
#     #         for layer in model.layers[1].layers:
#     #             if 'block3' not in layer.name and 'block4' not in layer.name:
#     #                 layer.trainable = False
#     #     return model
    
#     # model = build_classifier(backbone, num_classes, freeze_backbone=False)

#     # RESUMING TRAINING FROM PREVIOUS EPOCH MODEL CODE:

#     model = None  # Ensure model is not redefined if resuming
#     if fold == 1 and os.path.exists('/kaggle/input/capstone_movinet_model_and_weights/keras/default/1/movinet_classifier_fold_2.keras'):
#         model = tf.keras.models.load_model('/kaggle/input/capstone_movinet_model_and_weights/keras/default/1/movinet_classifier_fold_2.keras', compile=False)
#         print("Resumed training from Fold 2 .keras file")
#         initial_epoch = 4
#     else:
#         backbone = movinet.Movinet(model_id='a3')
#         backbone.build([batch_size, num_frames, resolution, resolution, 3])
#         model = movinet_model.MovinetClassifier(backbone=backbone, num_classes=12, dropout_rate=0.7)
#         model.build([batch_size, num_frames, resolution, resolution, 3])
#         initial_epoch = 0
    
#     # initial_learning_rate = 0.001
#     initial_learning_rate = 0.0005
#     total_train_steps = (len(train_paths_fold) // batch_size) * num_epochs
#     learning_rate = tf.keras.optimizers.schedules.CosineDecay(
#         initial_learning_rate, decay_steps=total_train_steps
#     )
#     model.compile(
#         loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
#         optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0),
#         metrics=['accuracy'],
#         jit_compile=True
#     )
    
#     # Callbacks
#     callbacks = [
#         tf.keras.callbacks.TensorBoard(log_dir=f'./logs/fold_{fold + 1}'),
#         tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
#         tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6)
#     ]
    
#     # Train model
#     history = model.fit(
#         train_data,
#         validation_data=val_data,
#         epochs=num_epochs,
#         initial_epoch=4,
#         validation_freq=1,
#         verbose=1,
#         callbacks=callbacks,
#         class_weight=class_weight_dict
#     )
    
#     # Evaluate and store metrics
#     val_loss, val_accuracy = model.evaluate(val_data, verbose=0)
#     fold_metrics.append({'fold': fold + 1, 'val_loss': val_loss, 'val_accuracy': val_accuracy})
#     print(f"Fold {fold + 1} - Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")
    
#     # Debugging: Check predictions
#     val_preds = model.predict(val_data, verbose=0)
#     val_preds_classes = np.argmax(val_preds, axis=1)
#     print(f"Fold {fold + 1} - Predicted class distribution:", {class_names[i]: sum(val_preds_classes == i) for i in range(num_classes)})  
#     print(f"Fold {fold + 1} - True class distribution:", {class_names[i]: sum(val_labels_fold == i) for i in range(num_classes)})  

#     cm = confusion_matrix(val_labels_fold, val_preds_classes)
#     plt.figure(figsize=(10, 8))
#     sns.heatmap(
#         cm / cm.sum(axis=1)[:, np.newaxis],  # Normalize by true class counts
#         annot=True,
#         fmt='.2f',
#         cmap='Blues',
#         xticklabels=class_names,
#         yticklabels=class_names
#     )
#     plt.title(f'Confusion Matrix - Fold {fold + 1}')
#     plt.xlabel('Predicted')
#     plt.ylabel('True')
#     plt.savefig(f'confusion_matrix_fold_{fold + 1}.png')  # Save for reference
#     plt.show()

    
#     # Save model
#     save_metrics(fold_metrics)
#     model.save(f"movinet_classifier_fold_{fold + 1}.keras")
#     model.save_weights(f"movinet_classifier_weights_fold_{fold + 1}.weights.h5")

#     del model, train_data, val_data
#     tf.keras.backend.clear_session()
#     gc.collect()
#     !nvidia-smi

# # Average metrics
# avg_val_loss = np.mean([m['val_loss'] for m in fold_metrics])
# avg_val_accuracy = np.mean([m['val_accuracy'] for m in fold_metrics])
# print(f"\nAverage across {k} folds - Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")
train_paths, val_paths, train_labels, val_labels = train_test_split(
    all_paths, all_labels, test_size=0.3, stratify=all_labels, random_state=42
)

print(f"\nFold 1/1")
print(f"Training clips: {len(train_paths)}")
print(f"Validation clips: {len(val_paths)}")
print("Validation class distribution:", {class_names[i]: sum(val_labels == i) for i in range(num_classes)})
print("Validation labels (all):", val_labels.tolist())
print("Unique validation labels:", np.unique(val_labels))

# Create datasets
train_data = create_dataset_from_paths(
    train_paths, train_labels, batch_size, class_names, augment=True, frame_stride=2
)
val_data = create_dataset_from_paths(
    val_paths, val_labels, batch_size, class_names, augment=False, frame_stride=2
)

print(f"Visualizing validation clips for fold 1")
visualize_clips(val_data, class_names, val_paths)

# Compute class weights
class_weights = class_weight.compute_class_weight(
    'balanced', classes=np.unique(train_labels), y=train_labels
)
class_weight_dict = dict(enumerate(class_weights))
if 11 in class_weight_dict:  # Reduce Yellow card weight
    class_weight_dict[11] *= 0.5
print(f"Class weights for fold 1:", class_weight_dict)

# Model setup
tf.keras.backend.clear_session()
gc.collect()

# Rebuild model and load weights
backbone = movinet.Movinet(model_id='a3')
backbone.build([batch_size, num_frames, resolution, resolution, 3])
model = movinet_model.MovinetClassifier(
    backbone=backbone,
    num_classes=num_classes,
    dropout_rate=0.7
)
model.build([batch_size, num_frames, resolution, resolution, 3])
if os.path.exists('/kaggle/input/capstone-thirditeration-model/keras/default/1/fold_1_epoch_4.weights.h5'):
    model.load_weights('/kaggle/input/capstone-thirditeration-model/keras/default/1/fold_1_epoch_4.weights.h5')
    print("Resumed training from Fold 1 .weights.h5 file")
    # initial_epoch = 4  # Resume from epoch 4
else:
    print("No weights file found, starting from scratch")
    # initial_epoch = 0

initial_learning_rate = 0.0005
learning_rate = tf.keras.optimizers.schedules.CosineDecay(
    initial_learning_rate, decay_steps=total_train_steps
)
model.compile(
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate, clipnorm=1.0),
    metrics=['accuracy'],
    jit_compile=True
)

# Callbacks
callbacks = [
    tf.keras.callbacks.TensorBoard(log_dir=f'./logs/fold_1'),
    tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True, verbose=1),
    tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, verbose=1, min_lr=1e-6),
    tf.keras.callbacks.ModelCheckpoint(filepath=f"iter_4_epoch_{{epoch}}.weights.h5", save_weights_only=True, save_best_only=False, verbose=1)
]

# Train model
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=num_epochs,
    validation_freq=1,
    verbose=1,
    callbacks=callbacks,
    class_weight=class_weight_dict
)

# Evaluate and store metrics
val_loss, val_accuracy = model.evaluate(val_data, verbose=0)
fold_metrics.append({'fold': 1, 'val_loss': val_loss, 'val_accuracy': val_accuracy})
print(f"Fold 1 - Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.4f}")

# Debugging: Check predictions
val_preds = model.predict(val_data, verbose=0)
val_preds_classes = np.argmax(val_preds, axis=1)
print(f"Fold 1 - Predicted class distribution:", {class_names[i]: sum(val_preds_classes == i) for i in range(num_classes)})
print(f"Fold 1 - True class distribution:", {class_names[i]: sum(val_labels == i) for i in range(num_classes)})

cm = confusion_matrix(val_labels, val_preds_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm / cm.sum(axis=1)[:, np.newaxis],
    annot=True,
    fmt='.2f',
    cmap='Blues',
    xticklabels=class_names,
    yticklabels=class_names
)
plt.title(f'Confusion Matrix - Fold 1')
plt.xlabel('Predicted')
plt.ylabel('True')
plt.savefig(f'confusion_matrix_epochs_50.png')
plt.show()

# Save model
save_metrics(fold_metrics)
model.save(f"movinet_classifier_epoch_10.keras")
model.save_weights(f"movinet_classifier_weights_epoch_10.weights.h5")

del model, train_data, val_data
gc.collect()
!nvidia-smi

# Average metrics
avg_val_loss = np.mean([m['val_loss'] for m in fold_metrics])
avg_val_accuracy = np.mean([m['val_accuracy'] for m in fold_metrics])
print(f"\nAverage across 1 folds - Val Loss: {avg_val_loss:.4f}, Val Accuracy: {avg_val_accuracy:.4f}")


Fold 1/1
Training clips: 2585
Validation clips: 1109
Validation class distribution: {'Corner': 69, 'Direct free-kick': 95, 'Foul': 79, 'Goal': 100, 'Indirect free-kick': 78, 'Kick-off': 106, 'Offside': 80, 'Shots off target': 105, 'Shots on target': 107, 'Substitution': 101, 'Throw-in': 80, 'Yellow card': 109}
Validation labels (all): [7, 7, 8, 2, 11, 0, 0, 7, 9, 1, 5, 3, 7, 1, 6, 0, 6, 5, 8, 2, 8, 10, 10, 2, 8, 3, 6, 6, 9, 6, 4, 0, 8, 1, 6, 9, 10, 8, 7, 0, 5, 10, 11, 1, 4, 3, 0, 11, 8, 11, 1, 3, 8, 3, 11, 11, 6, 9, 6, 8, 8, 5, 11, 1, 8, 6, 5, 0, 11, 6, 8, 2, 10, 8, 11, 11, 1, 5, 11, 9, 11, 0, 7, 5, 5, 7, 7, 4, 3, 7, 11, 1, 9, 5, 8, 5, 9, 4, 2, 1, 9, 3, 11, 2, 6, 5, 1, 6, 8, 8, 5, 7, 8, 9, 11, 11, 0, 11, 1, 10, 5, 5, 6, 11, 4, 7, 11, 7, 8, 7, 5, 4, 7, 7, 0, 9, 5, 5, 9, 1, 0, 8, 3, 0, 4, 11, 9, 8, 11, 3, 3, 0, 9, 2, 4, 7, 4, 9, 8, 8, 2, 2, 6, 10, 4, 7, 2, 0, 7, 11, 3, 7, 4, 6, 3, 9, 5, 10, 6, 0, 6, 5, 5, 11, 2, 8, 5, 1, 3, 5, 1, 5, 7, 3, 5, 11, 3, 11, 2, 0, 7, 11, 4, 9, 2, 8, 8, 5, 0, 

I0000 00:00:1745011945.074108   68357 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 15513 MB memory:  -> device: 0, name: Tesla P100-PCIE-16GB, pci bus id: 0000:00:04.0, compute capability: 6.0


Visualizing validation clips for fold 1
Sample validation labels: [3 1 8 3]
Sample validation paths: ['/kaggle/input/capstone-dataset-updated/split_data_2/train/Shots off target/aug_6102_Shots off target_149.mp4', '/kaggle/input/capstone-dataset-updated/split_data_2/train/Shots off target/Shots off target_184.mp4', '/kaggle/input/capstone-dataset-updated/split_data_2/train/Shots on target/Shots on target_38.mp4', '/kaggle/input/capstone-dataset-updated/split_data_2/test/Foul/Foul_188.mp4']
Class weights for fold 1: {0: 1.329732510288066, 1: 0.9791666666666666, 2: 1.1644144144144144, 3: 0.9245350500715308, 4: 1.1901473296500922, 5: 0.8756775067750677, 6: 1.1519607843137254, 7: 0.8864883401920439, 8: 0.8616666666666667, 9: 0.9127824858757062, 10: 1.1519607843137254, 11: 0.4223856209150327}
Resumed training from Fold 1 .weights.h5 file
Epoch 1/6


I0000 00:00:1745012075.657976   68420 service.cc:152] XLA service 0x7e9e98255ce0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1745012075.658018   68420 service.cc:160]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1745012100.588023   68420 cuda_dnn.cc:529] Loaded cuDNN version 90300
2025-04-18 21:35:02.839311: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Constant folding an instruction is taking > 1s:

  %reduce-window.10147 = f32[5,16,56,56,16]{4,3,2,1,0} reduce-window(f32[5,16,112,112,16]{4,3,2,1,0} %broadcast.10136, f32[] %constant.10137), window={size=1x1x3x3x1 stride=1x1x2x2x1 pad=0_0x0_0x0_1x0_1x0_0}, to_apply=%region_1.10143, metadata={op_type="AvgPool3D" op_name="movinet_classifier/movinet/block0_layer0/bneck/skip/skip_pool/AvgPool3D"}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some g

517/517 [==============================] - ETA: 0s - loss: 1.9545 - accuracy: 0.2662

2025-04-18 21:44:19.862309: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Constant folding an instruction is taking > 8s:

  %reduce-window.1704 = f32[5,16,56,56,16]{4,3,2,1,0} reduce-window(f32[5,16,112,112,16]{4,3,2,1,0} %broadcast.1693, f32[] %constant.1694), window={size=1x1x3x3x1 stride=1x1x2x2x1 pad=0_0x0_0x0_1x0_1x0_0}, to_apply=%region_1.1700, metadata={op_type="AvgPool3D" op_name="movinet_classifier/movinet/block0_layer0/bneck/skip/skip_pool/AvgPool3D"}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2025-04-18 21:44:56.416437: E external/local_xla/xla/service/slow_operation_alarm.cc:140] The operation took 44.55701


Epoch 1: saving model to iter_4_epoch_1.weights.h5
517/517 [==============================] - 932s 1s/step - loss: 1.9545 - accuracy: 0.2662 - val_loss: 2.0828 - val_accuracy: 0.2543 - lr: 4.8343e-04
Epoch 2/6
517/517 [==============================] - ETA: 0s - loss: 1.7722 - accuracy: 0.3164
Epoch 2: saving model to iter_4_epoch_2.weights.h5
517/517 [==============================] - 547s 1s/step - loss: 1.7722 - accuracy: 0.3164 - val_loss: 2.1693 - val_accuracy: 0.2678 - lr: 4.3581e-04
Epoch 3/6
517/517 [==============================] - ETA: 0s - loss: 1.5187 - accuracy: 0.4108
Epoch 3: saving model to iter_4_epoch_3.weights.h5
517/517 [==============================] - 498s 964ms/step - loss: 1.5187 - accuracy: 0.4108 - val_loss: 2.3897 - val_accuracy: 0.2642 - lr: 3.6347e-04
Epoch 4/6
517/517 [==============================] - ETA: 0s - loss: 1.1982 - accuracy: 0.5261
Epoch 4: ReduceLROnPlateau reducing learning rate to 0.0001380131288897246.

Epoch 4: saving model to iter_4_ep

2025-04-18 22:17:40.046336: E external/local_xla/xla/service/slow_operation_alarm.cc:73] Constant folding an instruction is taking > 32s:

  %reduce-window.1696 = f32[5,16,56,56,16]{4,3,2,1,0} reduce-window(f32[5,16,112,112,16]{4,3,2,1,0} %broadcast.1685, f32[] %constant.1686), window={size=1x1x3x3x1 stride=1x1x2x2x1 pad=0_0x0_0x0_1x0_1x0_0}, to_apply=%region_1.1692, metadata={op_type="AvgPool3D" op_name="movinet_classifier/movinet/block0_layer0/bneck/skip/skip_pool/AvgPool3D"}

This isn't necessarily a bug; constant-folding is inherently a trade-off between compilation time and speed at runtime. XLA has some guards that attempt to keep constant folding from taking too long, but fundamentally you'll always be able to come up with an input program that takes a long time.

If you'd like to file a bug, run with envvar XLA_FLAGS=--xla_dump_to=/tmp/foo and attach the results.
2025-04-18 22:17:53.698953: E external/local_xla/xla/service/slow_operation_alarm.cc:140] The operation took 45.6551

Fold 1 - Predicted class distribution: {'Corner': 43, 'Direct free-kick': 51, 'Foul': 104, 'Goal': 144, 'Indirect free-kick': 102, 'Kick-off': 77, 'Offside': 137, 'Shots off target': 4, 'Shots on target': 169, 'Substitution': 134, 'Throw-in': 127, 'Yellow card': 17}
Fold 1 - True class distribution: {'Corner': 69, 'Direct free-kick': 95, 'Foul': 79, 'Goal': 100, 'Indirect free-kick': 78, 'Kick-off': 106, 'Offside': 80, 'Shots off target': 105, 'Shots on target': 107, 'Substitution': 101, 'Throw-in': 80, 'Yellow card': 109}
Fri Apr 18 22:21:15 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 560.35.03              Driver Version: 560.35.03      CUDA Version: 12.6     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GP

In [9]:
import os
import zipfile

# List files to include
files_to_zip = ['confusion_matrix_fold_1.png', 'movinet_classifier_weights_fold_1.weights.h5','movinet_classifier_fold_1.keras']
# files_to_zip = ['confusion_matrix_fold_1.png', 'confusion_matrix_fold_2.png', 'movinet_classifier_fold_1.keras', 'movinet_classifier_fold_2.keras','movinet_classifier_weights_fold_1.weights.h5', 'movinet_classifier_weights_fold_2.weights.h5']
output_zip = 'outputs.zip'

# Create a zip file
with zipfile.ZipFile(output_zip, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for file in files_to_zip:
        if os.path.exists(file):
            zipf.write(file)
        else:
            print(f"Warning: {file} not found")

# Provide download link
from IPython.display import FileLink
FileLink(output_zip)

/kaggle/working/outputs.zip

# **END OF LATEST CODE**